<a href="https://colab.research.google.com/github/YunhoCha/2024-DH/blob/main/gb_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1_웹 스크래핑

시작전 왼쪽에 data 폴더와 result 폴더을 만들고 data 폴더에 메타데이터 파일을 꼭 넣을 것

# 2_필요한 도구와 라이브러리 설치

In [ ]:
import ssl
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 3_url 분석

In [ ]:
# "개벽" 기사가 서비스되는 웹페이지 주소 1개를 불러와서 url 변수에 저장하기
# 이 url에서 기사별 id가 있는 부분을 경계로 그 이전과 이후를 분리하기

url = 'https://db.history.go.kr/modern/level.do?levelId=ma_013_0010_0001'
url_s = url.split('ma_013_0010_0001')
print(url)
print(url_s)

# 4_기사정보 다운로드

사전 작업
- 국편 한국사데이터베이스/개벽에서 기사정보 다운로드
- '개벽_기사정보.txt'로 저장

In [ ]:
# 이름 조심하기
gb_info = 'data/근현대잡지자료 수정본.txt'

In [ ]:
df = pd.read_csv(gb_info, delimiter='^', encoding='utf-8')
df

In [ ]:
df.info()

In [ ]:
# '발행일' 열이 공백이면 기사내용이 포함되지 않은 행이라는 뜻
df1 = df[['자료ID', '기사제목', '필자', '기사형태', '발행일', 'URL']]
df1

In [ ]:
# 결측치 확인, 발행일열이 공백인 77개의 행을 없앨 필요가 있음
df1.isna().sum()

In [ ]:
# 발행일 열이 공백인 행을 보기
df1[df1['발행일'].isna()][:30]

In [ ]:
# 요약통계
df1.describe()

In [ ]:
# 값 파악
df1['발행일'].value_counts().sort_index()[60:90]

In [ ]:
# 결측치 제거
df1 = df1.copy()
df1 = df1.dropna(subset = ['발행일'])  # 발행일 열이 비어있는 곳을 없애기
df1

In [ ]:
# 1930년대 개벽 제외
df1['year'] = df1['발행일'].apply(lambda x: int(x[:4]))
df2 = df1.query('year < 1930')
df2

In [ ]:
df2.info() # 자료id가 2257개 임을 알 수 있음

In [ ]:
df2.isna().sum() # 다시한번 결측치를 확인하니 발행일 쪽 결측치가 0이 됨

In [ ]:
# 자료ID 길이 확인 <- 크로스 체크
df2['자료ID'].str.len().value_counts()
# 자료id가 글자길이가 16인 것이 2257개임을 알 수 있음
# 만약 글자길이가 다른 id가 존재한다면 그것도 결측치로 제거할 수 있음(밑 코드 이용)

In [ ]:
# df2 = df1[df1['자료ID'].str.len() != 17] # 길이가 17인 자료id 행 삭제
# df2

# 5_기사 id를 이용한 원문 스크래핑
- url_s 변수를 이용하여 기사 ID를 포함한 웹페이지 주소를 생성
- 이 주소를 이용하여 웹 페이지에 접속하고, BeautifulSoup을 사용하여 원하는 부분의 데이터를 추출
- 국편위 개별 사료 페이지로 가서 Ctrl + Shift + i 를 누르기, html에서 텍스트가 어느 위치에 있는지 알아보기(실습)

In [ ]:
# 기사 id를 모두 리스트로 변환

id_List = df2['자료ID'].to_list()
print(len(id_List))
id_List[:10]

In [ ]:
# 본문 가져오기 사용자 함수 작성

def get_contents(url):
    nm = 1
    results = []  # 결과를 저장할 리스트 생성

    # SSL 인증서 확인을 건너뛰도록 설정
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    for ids in id_List[:5]: # 샘플로 5개 기사만 추출, 숫자를 아까 자료 id 전체 크기로 바꾸면 모든 사료를 스크래핑하는 것
        N_url = url_s[0] + ids + url_s[-1]
        # SSL 인증서 확인을 건너뛰는 urlopen 사용
        webpage = urlopen(N_url, context=ctx)

        bsobj = BeautifulSoup(webpage.read(), 'html.parser')
        List1 = bsobj.findAll('div', {'id': 'cont_view'})  # 이 부분을 국편위 사이트 개별 사료에서 html을 파악하여 위치를 넣는 곳

        for z in List1:
            z1 = z.get_text('\n', strip=True)
            results.append([ids, z1])  # 결과를 리스트에 추가

    # pandas DataFrame으로 변환
    result_df = pd.DataFrame(results, columns=['자료ID', 'Content'])
    return result_df

In [ ]:
contents_5_df = get_contents(url)
contents_5_df

In [ ]:
# 본문 내용 확인 (첫 행)
print(contents_5_df.loc[3,'Content'])

# 6_데이터 저장

In [ ]:
# 두 데이터프레임을 '자료ID' 기준으로 통합

combi_df = pd.merge(df2, contents_5_df, on='자료ID', how='inner')
combi_df

In [ ]:
# 저장
combi_df.to_excel('result/A1_raw5.xlsx', index=False)  # 인덱스를 저장하지 않도록 설정

# The End of Note

---

# 텍스트 전처리(형태소 분석)과 간단한 빈도 분석 실습